In [122]:
import pandas as pd
import os
from nltk.tokenize import word_tokenize
from pyvi import ViTokenizer, ViPosTagger
from nltk import pos_tag

In [123]:
path = ".\\Bahnar_dataset"
folder = "KH-CN"

READ DICTIONARY

In [124]:
dict_path = path + "\\Kriem-Common_PoS.xlsx"
dict_frame =pd.read_excel(dict_path)

dict_frame.head(5)

,No.,Bahnaric,Vietnamese,PoS tag
0,1,abĕn,váy,Noun
1,2,abŭt,băo,Verb
2,3,ada,vịt,Noun
3,4,adal adal,"cẩn thận, khe khẽ, thong thả",Adjective
4,5,adon,xếp,Verb


In [125]:
def push_word(dict, word, value):
    if word in dict:
        # Check for tag
        not_Contain = 1
        for i in dict[word][1]:
            if value[1] == i:
                not_Contain = 0
                break
        if not_Contain == 1:
            dict[word][1].append(value[1])
            dict[word][0].append(value[0])
            print(f"{word} with Tag: {len(dict[word][1])}")            
    else:
        dict[word] = [[], []]
        dict[word][0].append(value[0])
        dict[word][1].append(value[1])

In [126]:
dict = {}
for idx, str in enumerate(dict_frame['Bahnaric']):
    lst_word = str.split(',')
    contend = dict_frame.iloc[idx][2:4]
    for word in lst_word:
        word = word.strip()
        push_word(dict, word, contend)

ba with Tag: 2
bal with Tag: 2
bĕng with Tag: 2
bih with Tag: 2
bŏ with Tag: 2
blah with Tag: 2
hăp with Tag: 2
hlak with Tag: 2
kăp with Tag: 2
klaih with Tag: 2
kung with Tag: 2
luôn with Tag: 2
mơ̆u with Tag: 2
nhŭ with Tag: 2
sil with Tag: 2
sĭn with Tag: 2
sră with Tag: 2
tŏ with Tag: 2
drŏng with Tag: 2
hlak with Tag: 3
weh with Tag: 2
yôm with Tag: 2


READ DATASET

In [127]:
dataset = []
file_name = []
dataset = []
dataset_path = path + "\\" + folder
for obj in os.scandir(dataset_path):
    dataframe = pd.read_excel(dataset_path + "\\" + obj.name)
    dataset.append((obj.name, dataframe))

In [128]:
a, b = dataset[0]
print(a)
print(b.head(5))

KH-CN bai 03.xlsx
   ID                                         Tiếng Việt  \
0   1  Quyết định về việc phê duyệt Kế hoạch lựa chọn...   
1   2  Xây dựng mô hình ứng dụng tiến bộ khoa học côn...   
2   3   Căn cứ Luật Tổ chức HĐND và UBND ngày 26/11/2003   
3   4  Căn cứ Luật Đấu thầu số 43/2013/QH13 ngày 26/1...   
4   5  Căn cứ Nghị định số 63/2014/NĐ-CP ngày 26/6/20...   

                                          Tiếng BaNa  
0  Kuiê̆t đĭnh lơm dron gphê yuiêt Kê̆ hoach...  
1  Pơjing mô hinh ư̆ng yŭng tiê̆n 'bô̆ khoa h...  
2  Kăn kư̆ Luât tô chưk HNĐND xưm UBND năr 'b...  
3  Kăn kư̆ Luât đâ̆u thâu sô̆ puôn jĭt pê...  
4  Kăn kư̆ Nghĭ đĭnh sô̆ drâu jĭt pêng xơn...  


TOKENIZING

In [130]:
class Sequence_Info():
    def __init__(self, bana_token, bana_tag, vietnam_token, vietnam_tag, miscount) -> None:
        self.bana_token = bana_token
        self.bana_tag = bana_tag

        self.vietnam_token = vietnam_token
        self.vietnam_tag = vietnam_tag

        self.miscount = miscount

    def get_bana(self):
        return self.bana_token
    
    def get_vietnamese(self):
        return self.vietnam_token
    
    def get_miscount(self):
        return self.miscount
    
    def write_excel(self, fp):
        fp.write(f"{self.miscount}\n")
        

        token_str = self.bana_token[0]
        tag_str = '/'.join(self.bana_tag[0])
        for token, tag in zip(self.bana_token[1:], self.bana_tag[1:]):
            tag_str = tag_str + "," + '/'.join(tag)
            token_str = token_str + "," + token

        fp.write(f"{token_str}\n")
        fp.write(f"{tag_str}\n")

        fp.write(f"{','.join(self.vietnam_token)}\n")
        fp.write(f"{','.join(self.vietnam_tag)}\n")
        

In [131]:
def token_tag_vietnam(sequence):
    token, tag = ViPosTagger.postagging(ViTokenizer.tokenize(sequence))
    return token, tag

def token_tag_bahnar(sequence):
    token_list = []
    tag_list = []

    words = word_tokenize(sequence)
    miscount = 0
        
    i = 0
    while(i < len(words)):
        j = 1
        while (i + j <= len(words)):
            if ' '.join(words[i : i + j]) in dict:
                j += 1
            else:
                break
            
        # In case this word is in dict => write into a file and then mark the pos tag for it
        if j > 1:
            token = ' '.join(words[i : i + j - 1])

            token_list.append(token)
            tag_list.append(dict[token][1])
            
        # In case the token is not in dict => write the token and mark it
        else:
            miscount += 1
            token_list.append(words[i])
            tag_list.append(" ")

        i += j

    return token_list, tag_list, miscount
    

In [132]:
def sort_by_missword(info):
    return info.miscount

In [133]:
contain_file = f"{folder}_info"
if not os.path.isdir(f"{path}\\{contain_file}"):
    os.makedirs(f"{path}\\{contain_file}")

for file_name, contend in dataset:
    sentence_lst = []
    for idx, paragraph in contend.iterrows():
        if pd.notna(paragraph[2]) and pd.notna(paragraph[1]):

            viet_token, viet_tag = token_tag_vietnam(paragraph[1])

            bana_token, bana_tag, miscount = token_tag_bahnar(paragraph[2])

            seq_info = Sequence_Info(bana_token, bana_tag, viet_token, viet_tag, miscount)
            sentence_lst.append(seq_info)

    # Sorting the sequence info following the number of missing words
    sentence_lst.sort(key=sort_by_missword)

    # Write in the csv file
    fstream = open(path + "\\" + contain_file + "\\" + file_name.split('.')[0] + ".csv", 'w+', encoding='utf-8')
    for i in sentence_lst:
        i.write_excel(fstream)
        fstream.write("\n")

    fstream.close()